# Collecting data from ThermoML xml files


## Import pkgs

In [ ]:
import os
from ThermoML import ThermoMLBuilder
import polars as pl
import numpy as np
import glob

thermoml_schema.py generator:


In [ ]:
# pyxbgen -u ThermoML/ThermoML.xsd -m ThermoML/thermoml_schema

In [ ]:
def RetrieveFiles():
    """
    Outputs a list of files path from ThermoML archive.

    Outputs
    ----------
    files: list(str)
        list of paths for each reference file
    """

    dir = os.getcwd()
    files = [os.path.join(dir, file) for file in glob.glob("ThermoML/*/*.xml")]
    return files

## Raw data


Collecting data from all files


In [ ]:
files = RetrieveFiles()
files[0]

In [ ]:
data = ThermoMLBuilder.build_dataset(files, "raw", executor="process", max_workers=12)

## Check error files

In [ ]:
content = ThermoMLBuilder._worker_parse("d:\\ThermoMLDataCollector\\ThermoML\\10.1016\\j.fluid.2018.07.020.xml")


In [ ]:
content and content[0]

(Optional) remove empty RegNum from files that presented this error

In [ ]:
from pathlib import Path
ERROR_LOG = Path("./data/raw/errorLOG.txt").resolve()

In [ ]:
from ThermoML import process_log
_ = process_log.process_all(ERROR_LOG)